## **MIDI Factory**

#### Import Libraries

In [1]:
import os

import xlwings as xw
from midiArabia import MidiArabia

### Constant 

In [ ]:
path_midi = "./midi/test.mid"

In [ ]:
song = MidiArabia(path_midi= path_midi,detect_rhythms=False)
dataFrame = song.midi_to_dataFrame()

### Access Excel sheet

In [ ]:
file_name = "test.xlsx"
sheet_name = "sheet1"

sheet_exist = False
# Check if the file exists
if not os.path.exists(file_name):
    # Create an empty file
    wb = xw.Book()
    wb.save(file_name)
    wb.close()
    print(f"{file_name} was successfully created.")

## open excel file 
workbook = xw.Book(file_name)

for sheet in workbook.sheets:
    sheet_exist = sheet_name == sheet.name
    if(sheet_exist) : break

# Check if 'Sheet1' exists
if not sheet_exist:
    workbook.sheets.add(sheet_name)
    print(f"{sheet_name} sheet was successfully created.")

### Update Excel File

In [ ]:
# Select the worksheet
worksheet = workbook.sheets[sheet_name]

# Delete the existing table (if it exists)
if worksheet.tables:
    worksheet.range(worksheet.tables[0].range.address).api.Delete()

# Add a new table to the worksheet
table = worksheet.tables.add(source=worksheet.range('A1'), name='MyTable')

# Update the table with the new data
table.update(dataFrame)

# Show the updated range of cells in a new window
table.range.api.Select()

# workbook.close()


True